In [31]:
from project.dataset import Dataset
import pandas as pd
import nibabel as nib

In [32]:
ds = Dataset()

### Load all cases


generate slice count for each case using the list comprehension `slices`


In [33]:
mri = ds.load_raw_mri()
masks = ds.load_cmb_masks()
slices = [nib.load(x).get_fdata().shape[2] for x in mri]

pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1

create `standard_df` for the unprocessed data


In [34]:
standard_df = pd.DataFrame({
    'mri': mri,
    'masks': masks,
    'slices': slices
})

In [35]:
sample_mask = nib.load(masks[0]).get_fdata()
sample_mask

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [36]:
sample_mask.shape

(512, 512, 35)

In [37]:
sample_mask[:, :, 0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

function to generate all the target slices for each case


In [38]:
def generate_target_slice(mri, masks, slices):
    if len(mri) != len(masks):
        print(f'Unequal amount of mri cases to cmb masks\t{len(mri)} to {len(masks)}')
    if len(mri) != len(slices):
        print(f'Unequal amount of mri cases to case slice counts\t{len(mri)} to {len(slices)}')

    # ls = [(mri[i], masks[i], target) for i in range(len(mri)) for target in range(slices[i])]
    ls = []
    
    for i in range(len(mri)):
        mask_data = nib.load(masks[i]).get_fdata()
        has_microbleed_case = 1 if mask_data.max() > 0 else 0
            
        for target in range(slices[i]):
            has_microbleed_slice = 1 if mask_data[:, :, target].max() > 0 else 0
            ls.append((
                mri[i], 
                masks[i], 
                target, 
                has_microbleed_case, 
                has_microbleed_slice
            ))
            
    df = pd.DataFrame(ls, columns=[
        'mri',
        'masks', 
        'target', 
        'has_microbleed_case', 
        'has_microbleed_slice'
    ])
    # ls = [(case, target) for case, slices in zip(case, slices) for target in range(slices)]
    return df

In [39]:
df = generate_target_slice(mri, masks, slices)

pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1
INFO:nibabel.global:pixdim[0] (qfac) should be 1 (default) or -1; setting qfac to 1


In [40]:
df.sample(5)

,mri,masks,target,has_microbleed_case,has_microbleed_slice
2449,d:\Github\Thesis and ML Project\Dataset\VALDO_...,d:\Github\Thesis and ML Project\Dataset\VALDO_...,144,0,0
6916,d:\Github\Thesis and ML Project\Dataset\VALDO_...,d:\Github\Thesis and ML Project\Dataset\VALDO_...,3,1,0
661,d:\Github\Thesis and ML Project\Dataset\VALDO_...,d:\Github\Thesis and ML Project\Dataset\VALDO_...,84,0,0
7576,d:\Github\Thesis and ML Project\Dataset\VALDO_...,d:\Github\Thesis and ML Project\Dataset\VALDO_...,25,1,0
2412,d:\Github\Thesis and ML Project\Dataset\VALDO_...,d:\Github\Thesis and ML Project\Dataset\VALDO_...,107,0,0


This should return none because this is invalid


In [41]:
df[(df['has_microbleed_case'] == 0) & (df['has_microbleed_slice'] == 1)]

,mri,masks,target,has_microbleed_case,has_microbleed_slice


Check how many cases have microbleeds


In [42]:
len(df['masks'].drop_duplicates())

72

In [43]:
df[df['has_microbleed_case'] == 1]['masks'].nunique()

50

In [44]:
ar = df.groupby('mri').target.max()
ar

mri
d:\Github\Thesis and ML Project\Dataset\VALDO_dataset\mri_t2s_dir\sub-101_space-T2S_desc-masked_T2S.nii.gz    34
d:\Github\Thesis and ML Project\Dataset\VALDO_dataset\mri_t2s_dir\sub-102_space-T2S_desc-masked_T2S.nii.gz    34
d:\Github\Thesis and ML Project\Dataset\VALDO_dataset\mri_t2s_dir\sub-103_space-T2S_desc-masked_T2S.nii.gz    34
d:\Github\Thesis and ML Project\Dataset\VALDO_dataset\mri_t2s_dir\sub-104_space-T2S_desc-masked_T2S.nii.gz    34
d:\Github\Thesis and ML Project\Dataset\VALDO_dataset\mri_t2s_dir\sub-105_space-T2S_desc-masked_T2S.nii.gz    34
                                                                                                              ..
d:\Github\Thesis and ML Project\Dataset\VALDO_dataset\mri_t2s_dir\sub-323_space-T2S_desc-masked_T2S.nii.gz    38
d:\Github\Thesis and ML Project\Dataset\VALDO_dataset\mri_t2s_dir\sub-324_space-T2S_desc-masked_T2S.nii.gz    38
d:\Github\Thesis and ML Project\Dataset\VALDO_dataset\mri_t2s_dir\sub-325_space-T2S_desc-mas

In [45]:
arr = standard_df.groupby('mri').slices.max()
ar

mri
d:\Github\Thesis and ML Project\Dataset\VALDO_dataset\mri_t2s_dir\sub-101_space-T2S_desc-masked_T2S.nii.gz    34
d:\Github\Thesis and ML Project\Dataset\VALDO_dataset\mri_t2s_dir\sub-102_space-T2S_desc-masked_T2S.nii.gz    34
d:\Github\Thesis and ML Project\Dataset\VALDO_dataset\mri_t2s_dir\sub-103_space-T2S_desc-masked_T2S.nii.gz    34
d:\Github\Thesis and ML Project\Dataset\VALDO_dataset\mri_t2s_dir\sub-104_space-T2S_desc-masked_T2S.nii.gz    34
d:\Github\Thesis and ML Project\Dataset\VALDO_dataset\mri_t2s_dir\sub-105_space-T2S_desc-masked_T2S.nii.gz    34
                                                                                                              ..
d:\Github\Thesis and ML Project\Dataset\VALDO_dataset\mri_t2s_dir\sub-323_space-T2S_desc-masked_T2S.nii.gz    38
d:\Github\Thesis and ML Project\Dataset\VALDO_dataset\mri_t2s_dir\sub-324_space-T2S_desc-masked_T2S.nii.gz    38
d:\Github\Thesis and ML Project\Dataset\VALDO_dataset\mri_t2s_dir\sub-325_space-T2S_desc-mas

### Check if all slices match with the raw dataframe count


In [46]:
(ar == (arr - 1)).all()

True

### Export as metadata


In [47]:
df.to_csv('targets.csv', index=False)